In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Feature_calculation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/07 12:43:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark

In [2]:
# read_rerun_builds_info
import json
import pandas as pd
from IPython.display import display


def read_builds_info(file_name):
    with open(file_name, "r") as json_file:
        loaded_dict = pd.read_json(json_file)
        converted_dict = {
            tuple(
                key.replace("(", "").replace(")", "").replace(" ", "").split(",")
            ): value
            for key, value in loaded_dict.items()
        }

    builds_info = {}
    for key, value in converted_dict.items():
        converted_value = int(key[1])
        builds_info[(key[0].replace("'", ""), converted_value)] = value

    return builds_info

In [3]:
def preprocess_name(name: str):
    return name.replace(" ", "").replace("-", "_").replace(".", "_")

In [4]:
rerun_builds_info = read_builds_info("../data/data.json")

display(rerun_builds_info[("Ife8387a3bdb75717b896cf1067d2490084665e49", 1)])

requestors          [{'1': {'user': 'pradip.kadam001', 'build_stat...
computation_time                                               105799
status                                                        failure
bot                 {'0': {'powerkvm': {'status': 'failure', 'jobs...
waiting_time                                                     8791
code_review_time                                                    0
recheck_count                                                       1
Name: ('Ife8387a3bdb75717b896cf1067d2490084665e49', 1), dtype: object

In [22]:
no_rerun_builds_info = read_builds_info("../data/data_one_run_redefined.json")

display(no_rerun_builds_info[("I00031eb7e7d6cd9b9fe86b5c649ddd6f64630ffe", 2)])

build_requestor     {'user': 'noonedeadpunk', 'build_status': 'suc...
computation_time                                                93362
status                                                        success
bot                 {'0': {'zuul': {'status': 'success', 'jobs': [...
waiting_time                                                    10521
code_review_time                                                    0
recheck_count                                                       0
Name: ('I00031eb7e7d6cd9b9fe86b5c649ddd6f64630ffe', 2), dtype: object

In [21]:
# create file with all user reruns calls
import csv

file_path = "../data/user/rerun_requestors.csv"


rerun_requestors = []
nan_key_info = []

for key, rerun_build_info in rerun_builds_info.items():
    # 'requestors': [{'1': {'user': 'pradip.kadam001',
    # 'build_status': 'failed',
    # 'created_at': '2019-11-07 13:55:00.000000000'}}],
    rerun_build_requestors = rerun_build_info["requestors"]
    for rerun_build_requestor in rerun_build_requestors:
        for rerun_num, rerun_requestor in rerun_build_requestor.items():
            try:
                rerun_requestor["user"] = preprocess_name(rerun_requestor["user"])
                rerun_requestors.append(rerun_requestor)
            except:
                nan_key_info.append((key, rerun_num))

with open(file_path, "w", newline="") as file:
    # build_status is actually rerun_result
    fields_array = ["user", "build_status", "created_at"]
    writer = csv.DictWriter(file, fieldnames=fields_array)

    writer.writeheader()
    writer.writerows(rerun_requestors)

In [4]:
# process by hand
print(nan_key_info)

NameError: name 'nan_key_info' is not defined

In [50]:
# create file with all bots: bot_name, previous_status, status, started_at, computation_time

import csv

file_path = "../data/bots/bots_history_info_with_previous_state.csv"


bots_history_info = []

for key, rerun_build_info in rerun_builds_info.items():
    bots_info = rerun_build_info["bot"]
    for rerun_num, bots in bots_info.items():
        for bot_name, bot in bots.items():
            previous_status = "none"
            try:
                if rerun_num != "0":
                    previous_status = bots_info[str(int(rerun_num) - 1)][bot_name][
                        "status"
                    ]
            except:
                pass

            bots_history_info.append(
                {
                    "bot_name": preprocess_name(bot_name),
                    "status": bot["status"],
                    "previous_status": previous_status,
                    "started_at": bot["started_at"],
                    "computation_time": bot["computation_time"],
                }
            )

with open(file_path, "w", newline="") as file:
    # build_status is actually rerun_result
    fields_array = [
        "bot_name",
        "previous_status",
        "status",
        "started_at",
        "computation_time",
    ]
    writer = csv.DictWriter(file, fieldnames=fields_array)

    writer.writeheader()
    writer.writerows(bots_history_info)

In [52]:
# create file with all jobs: job_name, status, started_at, computation_time, voting

import csv

file_path = "../data/jobs/jobs_history_info.csv"


jobs_history_info = []

for key, rerun_build_info in rerun_builds_info.items():
    bots_info = rerun_build_info["bot"]
    for rerun_num, bots in bots_info.items():
        for bot_name, bot in bots.items():
            for job in bot["jobs"]:
                jobs_history_info.append(
                    {
                        "job_name": f"{preprocess_name(bot_name)}:{preprocess_name(job['name'])}",
                        "status": job["status"],
                        "started_at": bot["started_at"],
                        "computation_time": job["time_in_seconds"],
                        "voting": job["voting"],
                    }
                )

with open(file_path, "w", newline="") as file:
    # build_status is actually rerun_result
    fields_array = [
        "job_name",
        "status",
        "started_at",
        "computation_time",
        "voting",
    ]
    writer = csv.DictWriter(file, fieldnames=fields_array)

    writer.writeheader()
    writer.writerows(jobs_history_info)

In [23]:
# build features for each rerun
import pandas as pd
import math
from tqdm import tqdm

rerun_user_info_file_path = "../data/user/rerun_requestors.csv"
rerun_all_requestors_info = pd.read_csv(rerun_user_info_file_path)

bots_history_info_file_path = "../data/bots/bots_history_info_with_previous_state.csv"
bots_history_info = pd.read_csv(bots_history_info_file_path)

jobs_history_info_file_path = "../data/jobs/jobs_history_info.csv"
jobs_history_info = pd.read_csv(jobs_history_info_file_path)

rerun_features = []

DEFAULT_SUCCESS_RATIO = -1  # for bots and user

for key, rerun_build_info in tqdm(rerun_builds_info.items()):
    bots_info = rerun_build_info["bot"]
    rerun_requestors = rerun_build_info["requestors"]
    for rerun_num, bots in bots_info.items():
        if int(rerun_num) != 0:
            rerun_feature = dict()

            # Add label
            rerun_feature["label"] = 1  # 0 failure , 1 success

            bots_success_ratio = 1
            bots_flip_success_ratio = 1
            jobs_success_ratio = 1
            for bot_name, bot in bots.items():
                ### Calculate label ###
                if bot["status"] == "failure":
                    rerun_feature["label"] = 0
            try:
                for bot_name, bot in bots_info[str(int(rerun_num) - 1)].items():
                    processed_bot_name = preprocess_name(bot_name)
                    current_bot_history = bots_history_info[
                        (bots_history_info["bot_name"] == processed_bot_name)
                        & (bots_history_info["started_at"] <= bot["started_at"])
                    ]

                    for job in bot["jobs"]:
                        processed_job_name = f"{preprocess_name(bot_name)}:{preprocess_name(job['name'])}"
                        current_job_history = jobs_history_info[
                            (jobs_history_info["job_name"] == processed_job_name)
                            & (jobs_history_info["started_at"] <= bot["started_at"])
                        ]
                        ### Count job success ratio ###
                        job_success_ratio = len(
                            current_job_history[
                                current_job_history["status"] == "success"
                            ]
                        ) / len(current_job_history)
                        if jobs_success_ratio > job_success_ratio:
                            jobs_success_ratio = job_success_ratio

                    ### Count bot success ratio ###
                    bot_success_ratio = len(
                        current_bot_history[current_bot_history["status"] == "success"]
                    ) / len(current_bot_history)
                    if bots_success_ratio > bot_success_ratio:
                        bots_success_ratio = bot_success_ratio

                    ### Count bot flip success ratio ###
                    flip_from_failure_to_success = current_bot_history[
                        (current_bot_history["status"] == "success")
                        & (current_bot_history["previous_status"] == "failure")
                    ]
                    flip_from_success_to_failure = current_bot_history[
                        (current_bot_history["status"] == "failure")
                        & (current_bot_history["previous_status"] == "success")
                    ]
                    all_flip_cases_number = len(flip_from_success_to_failure) + len(
                        flip_from_failure_to_success
                    )
                    bot_flip_success_ratio = (
                        len(flip_from_failure_to_success) / all_flip_cases_number
                        if all_flip_cases_number
                        else DEFAULT_SUCCESS_RATIO
                    )
                    if bots_flip_success_ratio > bot_flip_success_ratio:
                        bots_flip_success_ratio = bot_flip_success_ratio
            except:
                print("Error")
                print(str(key + tuple(rerun_num)))
                print(str(int(rerun_num) - 1))
                print(bots_info)

            rerun_feature["bots_success_ratio"] = bots_success_ratio
            rerun_feature["bots_flip_success_ratio"] = bots_flip_success_ratio
            rerun_feature["jobs_success_ratio"] = jobs_success_ratio

            ### Count user success ratio ###
            rerun_build_requestor = rerun_requestors[int(rerun_num) - 1]
            for rerun_num, rerun_requestor in rerun_build_requestor.items():
                if type(rerun_requestor["user"]) == str:
                    username = preprocess_name(rerun_requestor["user"])
                    current_requestor_history = rerun_all_requestors_info[
                        (rerun_all_requestors_info["user"] == username)
                        & (
                            rerun_all_requestors_info["created_at"]
                            < rerun_requestor["created_at"]
                        )
                    ]
                    user_success_ratio = (
                        len(
                            current_requestor_history[
                                current_requestor_history["build_status"] == "success"
                            ]
                        )
                        / len(current_requestor_history)
                        if len(current_requestor_history)
                        else DEFAULT_SUCCESS_RATIO
                    )
                    rerun_feature["user_success_ratio"] = user_success_ratio
                else:
                    # TODO check with ICE
                    rerun_feature["user_success_ratio"] = DEFAULT_SUCCESS_RATIO
            rerun_feature["id"] = str(key + tuple(rerun_num))
            rerun_features.append(rerun_feature)

# write results to file
with open("../data/features/features.csv", "w", newline="") as file:
    # build_status is actually rerun_result
    fields_array = [
        "id",
        "user_success_ratio",
        "bots_success_ratio",
        "bots_flip_success_ratio",
        "jobs_success_ratio",
        "label",
    ]
    writer = csv.DictWriter(file, fieldnames=fields_array)

    writer.writeheader()
    writer.writerows(rerun_features)
print(rerun_features[:2])

  0%|          | 4/36494 [00:45<100:58:20,  9.96s/it]

Error
('I000bbb64cacbf2298179e2b9888a1808d514fbec', 9, '1')
0
{'0': {'XenProject-CI': {'status': 'failure', 'jobs': [{'name': 'dsvm-tempest-xen', 'link': 'http://logs.openstack.xenproject.org/60/465160/9/check/dsvm-tempest-xen/67c20e6', 'voting': True, 'status': 'failure', 'time_in_seconds': 35}], 'started_at': '2017-10-12 15:42:32.000000000', 'computation_time': 0}, 'powerkvm': {'status': 'success', 'jobs': [{'name': 'tempest-dsvm-full-xenial', 'link': 'http://dal05.objectstorage.softlayer.net/v1/AUTH_3d8e6ecb-f597-448c-8ec2-164e9f710dd6/pkvmci/nova/60/465160/9/check/tempest-dsvm-full-xenial/5dc1526/', 'voting': True, 'status': 'success', 'time_in_seconds': 3268}, {'name': 'grenade-dsvm-xenial', 'link': 'http://dal05.objectstorage.softlayer.net/v1/AUTH_3d8e6ecb-f597-448c-8ec2-164e9f710dd6/pkvmci/nova/60/465160/9/check/grenade-dsvm-xenial/49a8c52/', 'voting': False, 'status': 'success', 'time_in_seconds': 2494}], 'started_at': '2017-10-12 16:36:30.000000000', 'computation_time': 0}, 'e

  0%|          | 6/36494 [00:46<55:47:58,  5.51s/it] 

In [ ]:
jobs_history_info_file_path = "../data/jobs/jobs_history_info.csv"
jobs_history_info = pd.read_csv(jobs_history_info_file_path)

In [64]:
processed_job_name = "zuul:openstack_ansible_varstest_aio"
temp_date = "2020-04-17"

print(rerun_builds_info.head())

current_job_history = jobs_history_info[
    (jobs_history_info["job_name"] == processed_job_name)
    & (jobs_history_info["started_at"] < temp_date)
]

print(jobs_history_info.nunique())

print(
    len(current_job_history[current_job_history["status"] == "success"])
    / len(current_job_history)
)

AttributeError: 'dict' object has no attribute 'head'

In [57]:
# build features for each rerun
import pandas as pd
import math
from tqdm import tqdm

rerun_user_info_file_path = "../data/user/rerun_requestors.csv"
rerun_all_requestors_info = pd.read_csv(rerun_user_info_file_path)

bots_history_info_file_path = "../data/bots/bots_history_info_with_previous_state.csv"
bots_history_info = pd.read_csv(bots_history_info_file_path)

jobs_history_info_file_path = "../data/jobs/jobs_history_info.csv"
jobs_history_info = pd.read_csv(jobs_history_info_file_path)

rerun_features = []

DEFAULT_SUCCESS_RATIO = -1  # for bots and user

for key, rerun_build_info in tqdm(rerun_builds_info.items()):
    bots_info = rerun_build_info["bot"]
    rerun_requestors = rerun_build_info["requestors"]
    for rerun_num, bots in bots_info.items():
        if int(rerun_num) != 0:
            rerun_feature = dict()

            bots_success_ratio = 1
            bots_flip_success_ratio = 1
            jobs_success_ratio = 1
            for bot_name, bot in bots.items():
                if bot["status"] == "failure":
                    rerun_feature["label"] = 0

                processed_bot_name = preprocess_name(bot_name)
                current_bot_history = bots_history_info[
                    (bots_history_info["bot_name"] == processed_bot_name)
                    & (bots_history_info["started_at"] < bot["started_at"])
                ]

                for job in bot["jobs"]:
                    processed_job_name = (
                        f"{preprocess_name(bot_name)}:{preprocess_name(job['name'])}"
                    )
                    current_job_history = jobs_history_info[
                        (jobs_history_info["job_name"] == processed_job_name)
                        & (jobs_history_info["started_at"] < bot["started_at"])
                    ]
                    ### Count job success ratio ###
                    job_success_ratio = (
                        len(
                            current_job_history[
                                current_job_history["status"] == "success"
                            ]
                        )
                        / len(current_job_history)
                        if len(current_job_history)
                        else DEFAULT_SUCCESS_RATIO
                    )
                    if jobs_success_ratio > job_success_ratio:
                        jobs_success_ratio = job_success_ratio

                ### Count bot flip success ratio ###
                flip_from_failure_to_success = current_bot_history[
                    (current_bot_history["status"] == "success")
                    & (current_bot_history["previous_status"] == "failure")
                ]
                flip_from_success_to_failure = current_bot_history[
                    (current_bot_history["status"] == "failure")
                    & (current_bot_history["previous_status"] == "success")
                ]
                all_flip_cases_number = len(flip_from_success_to_failure) + len(
                    flip_from_failure_to_success
                )
                bot_flip_success_ratio = (
                    len(flip_from_failure_to_success) / all_flip_cases_number
                    if all_flip_cases_number
                    else DEFAULT_SUCCESS_RATIO
                )
                if bots_flip_success_ratio > bot_flip_success_ratio:
                    bots_flip_success_ratio = bot_flip_success_ratio

            rerun_feature["bots_flip_success_ratio"] = bots_flip_success_ratio
            rerun_feature["jobs_success_ratio"] = jobs_success_ratio

            rerun_feature["id"] = str(key + tuple(rerun_num))
            rerun_features.append(rerun_feature)

# write results to file
with open(
    "../data/features/job_success_and_bot_flip_ratio.csv", "w", newline=""
) as file:
    # build_status is actually rerun_result
    fields_array = ["id", "bots_flip_success_ratio", "jobs_success_ratio"]
    writer = csv.DictWriter(file, fieldnames=fields_array)

    writer.writeheader()
    writer.writerows(rerun_features)
print(rerun_features[:2])

  0%|          | 136/36494 [1:18:16<348:47:52, 34.54s/it]  


KeyboardInterrupt: 

In [10]:
from tqdm import tqdm
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from pyspark.sql.types import StringType, DoubleType

# Initialize Spark session
spark = SparkSession.builder.appName("RerunFeatures").getOrCreate()

# Load data into Spark DataFrames
rerun_user_info_file_path = "../data/user/rerun_requestors.csv"
rerun_all_requestors_info = spark.read.csv(rerun_user_info_file_path, header=True)

bots_history_info_file_path = "../data/bots/bots_history_info_with_previous_state.csv"
bots_history_info = spark.read.csv(bots_history_info_file_path, header=True)

jobs_history_info_file_path = "../data/jobs/jobs_history_info.csv"
jobs_history_info = spark.read.csv(jobs_history_info_file_path, header=True)

rerun_features = []

DEFAULT_SUCCESS_RATIO = -1  # for bots and user

for key, rerun_build_info in tqdm(rerun_builds_info.items()):
    bots_info = rerun_build_info["bot"]
    rerun_requestors = rerun_build_info["requestors"]
    for rerun_num, bots in bots_info.items():
        if int(rerun_num) != 0:
            rerun_feature = dict()

            # Add label
            rerun_feature["label"] = 1  # 0 failure , 1 success

            bots_success_ratio = 1
            bots_flip_success_ratio = 1
            jobs_success_ratio = 1
            for bot_name, bot in bots.items():
                ### Calculate label ###
                if bot["status"] == "failure":
                    rerun_feature["label"] = 0
            try:
                for bot_name, bot in bots_info[str(int(rerun_num) - 1)].items():
                    processed_bot_name = preprocess_name(bot_name)
                    current_bot_history = bots_history_info.where(
                        (col("bot_name") == processed_bot_name)
                        & (col("started_at") <= bot["started_at"])
                    )

                    # for job in bot["jobs"]:
                    #     processed_job_name = f"{preprocess_name(bot_name)}:{preprocess_name(job['name'])}"
                    #     current_job_history = jobs_history_info[
                    #         (jobs_history_info["job_name"] == processed_job_name)
                    #         & (jobs_history_info["started_at"] <= bot["started_at"])
                    #     ]
                    #     ### Count job success ratio ###
                    #     job_success_ratio = len(
                    #         current_job_history[
                    #             current_job_history["status"] == "success"
                    #         ]
                    #     ) / len(current_job_history)
                    #     if jobs_success_ratio > job_success_ratio:
                    #         jobs_success_ratio = job_success_ratio

                    ### Count bot success ratio ###
                    success_count = current_bot_history.filter(
                        col("status") == "success"
                    ).count()
                    total_count = current_bot_history.count()
                    bot_success_ratio = success_count / total_count
                    if bots_success_ratio > bot_success_ratio:
                        bots_success_ratio = bot_success_ratio

                    # ### Count bot flip success ratio ###
                    # flip_from_failure_to_success = current_bot_history[
                    #     (current_bot_history["status"] == "success")
                    #     & (current_bot_history["previous_status"] == "failure")
                    # ]
                    # flip_from_success_to_failure = current_bot_history[
                    #     (current_bot_history["status"] == "failure")
                    #     & (current_bot_history["previous_status"] == "success")
                    # ]
                    # all_flip_cases_number = len(flip_from_success_to_failure) + len(
                    #     flip_from_failure_to_success
                    # )
                    # bot_flip_success_ratio = (
                    #     len(flip_from_failure_to_success) / all_flip_cases_number
                    #     if all_flip_cases_number
                    #     else DEFAULT_SUCCESS_RATIO
                    # )
                    # if bots_flip_success_ratio > bot_flip_success_ratio:
                    #     bots_flip_success_ratio = bot_flip_success_ratio
            except:
                print("Error")
                print(str(key + tuple(rerun_num)))
                print(str(int(rerun_num) - 1))
                print(bots_info)

            rerun_feature["bots_success_ratio"] = bots_success_ratio
            # rerun_feature["bots_flip_success_ratio"] = bots_flip_success_ratio
            # rerun_feature["jobs_success_ratio"] = jobs_success_ratio

            ### Count user success ratio ###
            # rerun_build_requestor = rerun_requestors[int(rerun_num) - 1]
            # for rerun_num, rerun_requestor in rerun_build_requestor.items():
            #     if type(rerun_requestor["user"]) == str:
            #         username = preprocess_name(rerun_requestor["user"])
            #         current_requestor_history = rerun_all_requestors_info[
            #             (rerun_all_requestors_info["user"] == username)
            #             & (
            #                 rerun_all_requestors_info["created_at"]
            #                 < rerun_requestor["created_at"]
            #             )
            #         ]
            #         user_success_ratio = (
            #             len(
            #                 current_requestor_history[
            #                     current_requestor_history["build_status"] == "success"
            #                 ]
            #             )
            #             / len(current_requestor_history)
            #             if len(current_requestor_history)
            #             else DEFAULT_SUCCESS_RATIO
            #         )
            #         rerun_feature["user_success_ratio"] = user_success_ratio
            #     else:
            #         # TODO check with ICE
            #         rerun_feature["user_success_ratio"] = DEFAULT_SUCCESS_RATIO
            rerun_feature["id"] = str(key + tuple(rerun_num))
            rerun_features.append(rerun_feature)

# Once you have computed the rerun_features as Spark DataFrame, you can write it to a CSV file
rerun_features_df = spark.createDataFrame(
    rerun_features
)  # Convert your rerun_features list to a Spark DataFrame

# Write results to file
rerun_features_df.write.csv(
    "../data/features/feature_build_success_ratio.csv", header=True
)

# Stop Spark session
spark.stop()

  0%|          | 4/36494 [00:04<11:50:49,  1.17s/it]

Error
('I000c6170d2f46be6b7c41f42fb32357b0f7142f8', 1, '2')
1
{'0': {'zuul': {'status': 'failure', 'jobs': [{'name': 'openstack-tox-docs', 'link': 'http://logs.openstack.org/00/613100/1/check/openstack-tox-docs/4dd2bbf/html/', 'voting': True, 'status': 'success', 'time_in_seconds': 186}, {'name': 'openstack-tox-pypy', 'link': 'http://logs.openstack.org/00/613100/1/check/openstack-tox-pypy/da76913/', 'voting': False, 'status': 'success', 'time_in_seconds': 246}, {'name': 'openstack-tox-pep8', 'link': 'http://logs.openstack.org/00/613100/1/check/openstack-tox-pep8/9fada25/', 'voting': True, 'status': 'success', 'time_in_seconds': 174}, {'name': 'openstack-tox-py27', 'link': 'http://logs.openstack.org/00/613100/1/check/openstack-tox-py27/d7a3a72/', 'voting': True, 'status': 'success', 'time_in_seconds': 272}, {'name': 'openstack-tox-py35', 'link': 'http://logs.openstack.org/00/613100/1/check/openstack-tox-py35/6d314ac/', 'voting': True, 'status': 'success', 'time_in_seconds': 207}, {'name

  0%|          | 31/36494 [00:28<8:51:30,  1.14it/s] 

Error
('I00637c94a6f93df736c78346af283eac16702f9f', 2, '2')
1
{'0': {'purestorage-cinder-ci': {'status': 'failure', 'jobs': [{'name': 'PureISCSIDriver-tempest-dsvm-bionic-aio-multipath-chap', 'link': 'http://openstack-logs.purestorage.com/39/745439/2/check/PureISCSIDriver-tempest-dsvm-bionic-aio-multipath-chap/2fcb577', 'voting': True, 'status': 'failure', 'time_in_seconds': 294}, {'name': 'PureFCDriver-tempest-dsvm-bionic-aio-multipath', 'link': 'http://openstack-logs.purestorage.com/39/745439/2/check/PureFCDriver-tempest-dsvm-bionic-aio-multipath/461de6b', 'voting': True, 'status': 'failure', 'time_in_seconds': 293}], 'started_at': '2020-08-10 07:33:49.000000000', 'computation_time': 0}, 'datera-ci': {'status': 'failure', 'jobs': [], 'started_at': '2020-08-10 07:37:14.000000000', 'computation_time': 0}, 'vmwareminesweeper': {'status': 'success', 'jobs': [{'name': 'dsvm-cinder', 'link': 'http://207.189.188.190/logs/39/745439/2/check-not-vote/ext-cinder-zuul/dd3c82a', 'voting': True, '

  0%|          | 97/36494 [01:24<15:05:49,  1.49s/it]ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.4/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
  0%|          | 98/36494 [01:27<20:36:40,  2.04s/it]

Error
('I00d6f13dd991074e4f45ada4d7cf4ccc0edbc7e1', 16, '4')
3
{'0': {'openstack-ironic-ci': {'status': 'failure', 'jobs': [{'name': 'dell-hw14G-UEFI-py2-tempest-dsvm-ironic-idrac', 'link': 'https://stash.dellemc-community.org/logs/73/583573/16/check/dell-hw14G-UEFI-py2-tempest-dsvm-ironic-idrac/54f1f33/', 'voting': True, 'status': 'failure', 'time_in_seconds': 110}, {'name': 'dell-hwDSS9000-tempest-dsvm-ironic-idrac', 'link': 'https://stash.dellemc-community.org/logs/73/583573/16/check/dell-hwDSS9000-tempest-dsvm-ironic-idrac/0bbafc4/', 'voting': True, 'status': 'failure', 'time_in_seconds': 111}, {'name': 'dell-hw13G-tempest-dsvm-ironic-idrac', 'link': 'https://stash.dellemc-community.org/logs/73/583573/16/check/dell-hw13G-tempest-dsvm-ironic-idrac/69bbf7c/', 'voting': True, 'status': 'failure', 'time_in_seconds': 126}, {'name': 'dell-hw14G-tempest-dsvm-ironic-idrac', 'link': 'https://stash.dellemc-community.org/logs/73/583573/16/check/dell-hw14G-tempest-dsvm-ironic-idrac/6eb2a83/', 

  0%|          | 104/36494 [01:35<13:53:49,  1.37s/it]

In [5]:
print(len(rerun_builds_info))

36494
